In [ ]:
import polars as pl


class Meal:
    def __init__(self, meal: str, day: int):
        self.day = day
        self.meal = meal
        self.ing_schema = {"ing": str, "unit": str, "val": float}
        self.schema = {"day": int, "name": str, **self.ing_schema}
        self.df = pl.DataFrame(schema=self.schema)

    def add_ingredient(self, name: str, unit: str, quantity: float):
        ing_df = pl.from_records(
            [[self.day, self.meal, name, unit, quantity]],
            schema=self.schema,
            orient="row",
        )

        self.df = pl.concat([self.df, ing_df])


In [28]:
import polars as pl

pl.Config().set_tbl_rows(99)
df = pl.DataFrame()


def construct_meal_tbl(
    day_num: int, meal_name: str, ingredients: dict[str, float]
) -> pl.DataFrame:
    df = pl.DataFrame()

    df = df.with_columns(pl.lit(day_num).alias("day"), pl.lit(meal_name).alias("meal"))

    ing_df = pl.DataFrame(
        [
            pl.Series(name="ing", values=ingredients.keys()),
            pl.Series(name="quant", values=ingredients.values()),
        ],
    )
    display(df)
    display(ing_df)
    df = df.join(
        ing_df,
        how="cross",
    )

    return df


burger_ings = [
    ["buns", "patties", 6],
    ["sliced_cheese", "slices", 8],
    ["tomato", 1],
    ["mayo (ml)", 120],
    ["patties", 4],
]
meal_1 = construct_meal_tbl(
    day_num=1,
    meal_name="burgers",
    ingredients={},
)
meal_2 = construct_meal_tbl(
    day_num=2,
    meal_name="pasta_stew",
    ingredients={"lentils (cans)": 2, "passata": 1, "carrots": 3, "onions": 2},
)


meatloaf_ings = [
    ["impossible mince", "packet", 2],
    ["carrots", "whole", 2],
    ["peas", "cup", 1],
    ["eggs", "whole", 2],
    ["brown mushrooms", "whole", 4],
    ["parsley", "sprig", 1],
    ["ketchup", "cup", 0.5],
    ["breadcrumbs", "cup", 2],
]

meal3 = Meal(meal="meatloaf", day=1)
for row in meatloaf_ings:
    meal3.add_ingredient(*row)

# meal_4 = construct_meal_tbl(
#     day_num=4,
#     meal_name="frittata",
#     ingredients=["eggs", "carrots", "leek", "potato", "peas", ""],
# )
meals = [
    meal_1,
    meal_2,
    #  meal_3
]
meals = pl.concat(meals, how="vertical")
meals


day,meal
i32,str
1,"""burgers"""


ing,quant
str,i64
"""buns""",6
"""sliced_cheese""",8
"""tomato""",1
"""mayo (ml)""",120
"""patties""",4


day,meal
i32,str
2,"""pasta_stew"""


ing,quant
str,i64
"""lentils (cans)""",2
"""passata""",1
"""carrots""",3
"""onions""",2


day,meal,ing,quant
i32,str,str,i64
1,"""burgers""","""buns""",6
1,"""burgers""","""sliced_cheese""",8
1,"""burgers""","""tomato""",1
1,"""burgers""","""mayo (ml)""",120
1,"""burgers""","""patties""",4
2,"""pasta_stew""","""lentils (cans)""",2
2,"""pasta_stew""","""passata""",1
2,"""pasta_stew""","""carrots""",3
2,"""pasta_stew""","""onions""",2
